In [1]:
import numpy as np
import pandas as pd

from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

import torch
import torch.nn as nn
import torch.nn.functional as F

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

import plotly.graph_objects as go
import matplotlib
matplotlib.use('agg')

In [2]:
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if len(data.shape) == 1 else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()

    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [("var%d(t-%d)" % (j + 1, i)) for j in range(n_vars)]

    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [("var%d(t)" % (j + 1)) for j in range(n_vars)]
        else:
            names += [("var%d(t+%d)" % (j + 1, i)) for j in range(n_vars)]

    agg = pd.concat(cols, axis=1)
    agg.columns = names

    if dropnan:
        agg.dropna(inplace=True)

    return agg

series = pd.read_csv(
    "../assets/datasets/time_series_solar.csv",
    parse_dates=["Datetime"],
    index_col="Datetime",
)["Incoming Solar"]

# Resample the data to daily frequency
series = series.resample("D").sum()

data = series_to_supervised(series, 3)
data.tail()

,var1(t-3),var1(t-2),var1(t-1),var1(t)
Datetime,,,,
2013-09-26,4113.6,2134.2,1250.2,1034.2
2013-09-27,2134.2,1250.2,1034.2,2182.3
2013-09-28,1250.2,1034.2,2182.3,3384.5
2013-09-29,1034.2,2182.3,3384.5,478.2
2013-09-30,2182.3,3384.5,478.2,2554.8


In [11]:
train, test = train_test_split(series, test_size=0.2, shuffle=False)

# differencing with .diff
train.diff(periods=1)

train_shifted = train.shift(periods=1)
train_diff = train - train_shifted

test_shifted = test.shift(periods=1)
test_diff = test - test_shifted
train_diff[:5], test_diff[:5]

(Datetime
 2007-10-01       NaN
 2007-10-02    2571.7
 2007-10-03    -855.1
 2007-10-04    -884.2
 2007-10-05    -875.1
 Freq: D, Name: Incoming Solar, dtype: float64,
 Datetime
 2012-07-19       NaN
 2012-07-20    1139.2
 2012-07-21     241.5
 2012-07-22    -627.1
 2012-07-23   -1586.7
 Freq: D, Name: Incoming Solar, dtype: float64)

In [15]:

scaler = MinMaxScaler(feature_range=(-1, 1))

train_diffnorm = scaler.fit_transform(train_diff.values.reshape(-1, 1))
test_diffnorm = scaler.transform(test_diff.values.reshape(-1, 1))

train_df = series_to_supervised(train_diffnorm, n_in=3, n_out=1).values
test_df = series_to_supervised(test_diffnorm, n_in=3, n_out=1).values
train_df[:5, :], test_df[:5, :]

(array([[ 0.50749214, -0.11870477, -0.12402237, -0.12235948],
        [-0.11870477, -0.12402237, -0.12235948,  0.46381843],
        [-0.12402237, -0.12235948,  0.46381843,  0.13297639],
        [-0.12235948,  0.46381843,  0.13297639,  0.04122506],
        [ 0.46381843,  0.13297639,  0.04122506,  0.03420803]]),
 array([[ 0.245724  ,  0.08168263, -0.07704115, -0.25239383],
        [ 0.08168263, -0.07704115, -0.25239383,  0.35313574],
        [-0.07704115, -0.25239383,  0.35313574, -0.30714129],
        [-0.25239383,  0.35313574, -0.30714129,  0.44295008],
        [ 0.35313574, -0.30714129,  0.44295008, -0.04270521]]))

In [4]:
X_train, y_train = train_df[:, :-1], train_df[:, -1]
X_test, y_test = test_df[:, :-1], test_df[:, -1]

X_train = torch.from_numpy(X_train).type(torch.Tensor)
X_test = torch.from_numpy(X_test).type(torch.Tensor)
y_train = torch.from_numpy(y_train).type(torch.Tensor).view(-1)
y_test = torch.from_numpy(y_test).type(torch.Tensor).view(-1)

X_train = X_train.view([X_train.shape[0], X_train.shape[1], 1])
X_test = X_test.view([X_test.shape[0], X_test.shape[1], 1])

In [5]:
class LSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim):
        super(LSTM, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).requires_grad_()
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).requires_grad_()
        out, (hn, cn) = self.lstm(x, (h0.detach(), c0.detach()))
        out = self.fc(out[:, -1, :])
        return out

In [6]:
model = LSTM(input_dim=1, hidden_dim=32, output_dim=1, num_layers=2)

loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

epochs = 200

for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()

    out = model(X_train).reshape(-1)
    loss = loss_fn(out, y_train)
    loss.backward()
    optimizer.step()

    if epoch % 10 == 0:
        print(f"Epoch: {epoch}, Loss: {loss.item()}")

model.eval()
y_pred = model(X_test).reshape(-1)

y_diff = scaler.inverse_transform(y_pred.detach().numpy().reshape(-1, 1)).flatten()
y_original = y_diff + test_shifted.values[4:]

np.abs(y_original - test.values[4:]).mean()

Epoch: 0, Loss: 0.06836027652025223
Epoch: 10, Loss: 0.06272191554307938
Epoch: 20, Loss: 0.062220875173807144
Epoch: 30, Loss: 0.06198015436530113
Epoch: 40, Loss: 0.06148086115717888
Epoch: 50, Loss: 0.060912568122148514
Epoch: 60, Loss: 0.060099244117736816
Epoch: 70, Loss: 0.05888807773590088
Epoch: 80, Loss: 0.05720594897866249
Epoch: 90, Loss: 0.055333830416202545
Epoch: 100, Loss: 0.05331984534859657
Epoch: 110, Loss: 0.050960298627614975
Epoch: 120, Loss: 0.04947558417916298
Epoch: 130, Loss: 0.049422476440668106
Epoch: 140, Loss: 0.0491449348628521
Epoch: 150, Loss: 0.0490131638944149
Epoch: 160, Loss: 0.04888748377561569
Epoch: 170, Loss: 0.04877478629350662
Epoch: 180, Loss: 0.04867594316601753
Epoch: 190, Loss: 0.04858482629060745


853.2065070278343

In [13]:
pred = scaler.inverse_transform(np.concatenate([X_test.detach().numpy().squeeze(), y_pred.detach().numpy().reshape(-1,1)], axis=1))
fig = go.Figure()
fig.add_trace(go.Scatter(x=data.index[:train.shape[0]], y=data.iloc[:train.shape[0]]["var1(t)"], name="Incoming Solar"))
fig.add_trace(go.Scatter(x=data.index[train.shape[0]:], y=data.iloc[train.shape[0]:]["var1(t)"], name="Actual"))
fig.add_trace(go.Scatter(x=data.index[train.shape[0]:], y=y_original[:], name="Prediction"))

fig.show()

In [21]:
train_df[:, -1]

array([-0.12235948,  0.46381843,  0.13297639, ..., -0.52770265,
        0.56898253,  0.05756158])

In [24]:
pred = scaler.inverse_transform(np.concatenate([X_test.detach().numpy().squeeze(), y_pred.detach().numpy().reshape(-1,1)], axis=1))
fig = go.Figure()
fig.add_trace(go.Scatter(x=data.index[:train.shape[0]], y=train_df[:, -1], name="Incoming Solar"))
fig.add_trace(go.Scatter(x=data.index[train.shape[0]:], y=test_df[:, -1], name="Actual"))
fig.add_trace(go.Scatter(x=data.index[train.shape[0]:], y=y_pred.detach().numpy(), name="Prediction"))

fig.show()